In [2]:
#Import Library
library(tidyverse)

In [3]:
# Converts all factors to character class
unfactorize <- function(df){
  for(i in which(sapply(df, class) == "factor")) df[[i]] = as.character(df[[i]])
  return(df)
}
# Code from user "By0" at https://stackoverflow.com/questions/2851015/convert-data-frame-columns-from-factors-to-characters (line 14)

In [63]:
# Converts character to factor class
factorize <- function(df){
  for(i in which(sapply(df, class) == "character")) df[[i]] = as.factor(df[[i]])
  return(df)
}

In [4]:
# Get and view last meta dataset
letters  <- unfactorize(read.csv("20201130_AM_Meta2Merge.csv"))
glimpse(letters)

Rows: 922
Columns: 37
$ X.2                     <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…
$ X.1                     <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…
$ X                       <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…
$ docsequence             <int> 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3…
$ docid                   <chr> "S10003-D023", "S10003-D024", "S10003-D025", …
$ docyear                 <int> 1836, 1836, 1837, 1837, 1838, 1838, 1838, 183…
$ doctype                 <chr> "Letter", "Letter", "Letter", "Letter", "Lett…
$ allsubject              <chr> "Childbirth; Church attendance; Cities; Farms…
$ broadsubj               <chr> "Health; Religion; Communities; Relationships…
$ personalevent           <chr> NA, NA, NA, NA, NA, NA, NA, NA, "Physical ill…
$ wwritten                <chr> "Baltimore, MD; Maryland; United States; Mid-…
$ docauthorid             <chr> "per0022938", "per0022938", "per0022938", "pe…
$ docauthorname           <chr

In [5]:
letters$sinceMigration  <- NA
letters$sinceMigration  <- letters$docyear - letters$yearimmigration
summary(letters$sinceMigration)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
 -85.00    3.00    7.00   14.88   24.00   62.00     263 

In [6]:
sum(is.na(letters$yearimmigration)|is.na(letters$docyear))

[1] 263

In [9]:
deleteDocIDs <- subset(letters, sinceMigration < 0, 
       select = c("docauthorid", 
                  "docauthorname", 
                  "yearimmigration", 
                  "docyear", 
                  "docid"))

deleteDocIDs

,docauthorid,docauthorname,yearimmigration,docyear,docid
334,per0000624,"Stritecky, Marie Spacil, ?-1944",1913,1828,S530-D014
335,per0000624,"Stritecky, Marie Spacil, ?-1944",1913,1831,S530-D015
336,per0038008,"Harris, Critchlow, 1813-1899",1856,1854,S6210-D007
337,per0038008,"Harris, Critchlow, 1813-1899",1856,1854,S6210-D009
338,per0038008,"Harris, Critchlow, 1813-1899",1856,1854,S6210-D011
368,per0038009,"Harris, Sarah Stretch, 1818-1897",1856,1853,S6210-D043
849,per0038013,"Fosdal, Aanen, fl. 1895-1898",1895,1892,S9983-D013


In [10]:
These are the IDs to remove
docIDs  <- deleteDocIDs$docid
docIDs

[1] "S530-D014"  "S530-D015"  "S6210-D007" "S6210-D009" "S6210-D011"
[6] "S6210-D043" "S9983-D013"

In [16]:
# Check to make sure that additional rows with same ID won't be deleted.
rows = which(grepl(paste(docIDs,collapse="|"), letters$docid))
letters$docid[rows]

[1] "S530-D014"  "S530-D015"  "S6210-D007" "S6210-D009" "S6210-D011"
[6] "S6210-D043" "S9983-D013"

In [28]:
nrow(subset(letters, !(docid %in% docIDs)))
915+7

[1] 915

[1] 922

In [30]:
lettersFinal <- subset(letters, !(docid %in% docIDs))
glimpse(lettersFinal)

Rows: 915
Columns: 38
$ X.2                     <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…
$ X.1                     <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…
$ X                       <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…
$ docsequence             <int> 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3…
$ docid                   <chr> "S10003-D023", "S10003-D024", "S10003-D025", …
$ docyear                 <int> 1836, 1836, 1837, 1837, 1838, 1838, 1838, 183…
$ doctype                 <chr> "Letter", "Letter", "Letter", "Letter", "Lett…
$ allsubject              <chr> "Childbirth; Church attendance; Cities; Farms…
$ broadsubj               <chr> "Health; Religion; Communities; Relationships…
$ personalevent           <chr> NA, NA, NA, NA, NA, NA, NA, NA, "Physical ill…
$ wwritten                <chr> "Baltimore, MD; Maryland; United States; Mid-…
$ docauthorid             <chr> "per0022938", "per0022938", "per0022938", "pe…
$ docauthorname           <chr

In [31]:
originalIDs  <- letters$docid
finalIDs <- lettersFinal$docid
removedIDs <- originalIDs[!(originalIDs %in% finalIDs)]
length(removedIDs)
removedIDs

[1] 7

[1] "S530-D014"  "S530-D015"  "S6210-D007" "S6210-D009" "S6210-D011"
[6] "S6210-D043" "S9983-D013"

In [32]:
# .txt to all IDs
fileids <- paste(removedIDs, ".txt", sep="")
fileids

[1] "S530-D014.txt"  "S530-D015.txt"  "S6210-D007.txt" "S6210-D009.txt"
[5] "S6210-D011.txt" "S6210-D043.txt" "S9983-D013.txt"

In [33]:
# Confirm working directory
getwd()

[1] "/Users/alaynemoody/Dropbox/Thesis/North_American_Letters_Diaries_OralHistories"

In [34]:
# Set working directory
setwd("letters")

In [35]:
# What are the files in that directory?
list.files()

[1] "S10003-D023.txt" "S10003-D024.txt" "S10003-D025.txt" "S10003-D026.txt"
  [5] "S10003-D027.txt" "S10003-D028.txt" "S10003-D029.txt" "S10003-D030.txt"
  [9] "S10003-D031.txt" "S10003-D032.txt" "S10003-D033.txt" "S10003-D034.txt"
 [13] "S10003-D035.txt" "S10003-D036.txt" "S10003-D037.txt" "S10003-D038.txt"
 [17] "S10003-D040.txt" "S10003-D041.txt" "S10003-D043.txt" "S10003-D044.txt"
 [21] "S10003-D046.txt" "S10003-D047.txt" "S10003-D048.txt" "S10003-D050.txt"
 [25] "S10003-D051.txt" "S10003-D052.txt" "S10003-D053.txt" "S10003-D054.txt"
 [29] "S10003-D055.txt" "S10003-D056.txt" "S10003-D057.txt" "S10003-D058.txt"
 [33] "S10003-D059.txt" "S10003-D060.txt" "S10003-D061.txt" "S10003-D062.txt"
 [37] "S10003-D063.txt" "S10003-D064.txt" "S10003-D065.txt" "S10003-D066.txt"
 [41] "S10003-D067.txt" "S10003-D068.txt" "S10003-D069.txt" "S10003-D072.txt"
 [45] "S10003-D074.txt" "S10003-D075.txt" "S10003-D081.txt" "S10003-D082.txt"
 [49] "S10003-D083.txt" "S10003-D084.txt" "S10003-D085.txt" "S10003-D086.txt"
 [53] "S10003-D088.txt" "S10003-D089.txt" "S10003-D091.txt" "S10003-D093.txt"
 [57] "S10003-D094.txt" "S10003-D096.txt" "S10003-D097.txt" "S10003-D098.txt"
 [61] "S10003-D100.txt" "S10003-D101.txt" "S10003-D102.txt" "S10003-D103.txt"
 [65] "S10003-D104.txt" "S10003-D105.txt" "S10003-D106.txt" "S10003-D108.txt"
 [69] "S10003-D109.txt" "S10003-D110.txt" "S10003-D111.txt" "S10003-D112.txt"
 [73] "S10003-D113.txt" "S10003-D114.txt" "S10003-D116.txt" "S10003-D117.txt"
 [77] "S10003-D118.txt" "S10003-D119.txt" "S10003-D120.txt" "S10003-D121.txt"
 [81] "S10003-D122.txt" "S10003-D123.txt" "S10003-D124.txt" "S10003-D125.txt"
 [85] "S10003-D126.txt" "S10003-D127.txt" "S10003-D128.txt" "S10003-D129.txt"
 [89] "S10003-D130.txt" "S10003-D134.txt" "S10003-D135.txt" "S10003-D136.txt"
 [93] "S10003-D137.txt" "S10003-D138.txt" "S10003-D139.txt" "S10003-D140.txt"
 [97] "S10003-D141.txt" "S10003-D142.txt" "S10003-D143.txt" "S10003-D144.txt"
[101] "S10003-D145.txt" "S10003-D146.txt" "S10003-D147.txt" "S10003-D148.txt"
[105] "S10003-D149.txt" "S10003-D150.txt" "S10003-D151.txt" "S10003-D152.txt"
[109] "S10003-D153.txt" "S10003-D154.txt" "S10003-D155.txt" "S10003-D156.txt"
[113] "S10003-D157.txt" "S10003-D158.txt" "S10003-D159.txt" "S10003-D160.txt"
[117] "S10003-D161.txt" "S10003-D162.txt" "S10003-D163.txt" "S10003-D164.txt"
[121] "S10003-D165.txt" "S10003-D166.txt" "S10003-D167.txt" "S10003-D168.txt"
[125] "S10003-D169.txt" "S10003-D170.txt" "S10003-D171.txt" "S10003-D172.txt"
[129] "S10003-D173.txt" "S10003-D174.txt" "S10003-D175.txt" "S10003-D176.txt"
[133] "S10003-D177.txt" "S10003-D178.txt" "S10003-D179.txt" "S10003-D180.txt"
[137] "S1019-D002.txt"  "S1019-D004.txt"  "S1019-D005.txt"  "S1019-D006.txt" 
[141] "S1019-D007.txt"  "S1019-D008.txt"  "S1019-D009.txt"  "S1019-D010.txt" 
[145] "S1019-D011.txt"  "S1019-D012.txt"  "S1019-D013.txt"  "S1019-D014.txt" 
[149] "S1019-D015.txt"  "S1019-D016.txt"  "S1019-D017.txt"  "S1019-D018.txt" 
[153] "S1019-D019.txt"  "S1019-D020.txt"  "S1019-D021.txt"  "S1019-D022.txt" 
[157] "S1019-D023.txt"  "S1019-D024.txt"  "S1019-D025.txt"  "S1019-D026.txt" 
[161] "S1019-D027.txt"  "S1019-D028.txt"  "S1019-D029.txt"  "S1019-D030.txt" 
[165] "S1019-D031.txt"  "S1019-D032.txt"  "S1019-D033.txt"  "S1019-D034.txt" 
[169] "S1019-D035.txt"  "S1019-D036.txt"  "S1019-D037.txt"  "S1019-D038.txt" 
[173] "S1019-D039.txt"  "S1019-D040.txt"  "S1019-D041.txt"  "S1019-D042.txt" 
[177] "S1019-D043.txt"  "S1019-D044.txt"  "S1019-D045.txt"  "S1019-D046.txt" 
[181] "S1019-D047.txt"  "S1019-D048.txt"  "S1019-D049.txt"  "S1019-D050.txt" 
[185] "S1019-D051.txt"  "S1019-D052.txt"  "S1019-D053.txt"  "S1019-D054.txt" 
[189] "S1019-D055.txt"  "S1019-D056.txt"  "S1019-D057.txt"  "S1019-D058.txt" 
[193] "S11067-D002.txt" "S11067-D004.txt" "S11067-D005.txt" "S11067-D008.txt"
[197] "S11067-D010.txt" "S11067-D012.txt" "S11067-D014.txt" "S11067-D015.txt"
[201] "S11067-D017.txt" "S11067-D018.txt" "S11067-D020.txt" "S11067-D023.txt"
[205] "S11067-D024.txt" 

In [36]:
# How many files are there?
length(list.files())

[1] 922

In [37]:
# Delete the fileids
sapply(fileids, unlink)

S530-D014.txt  S530-D015.txt S6210-D007.txt S6210-D009.txt S6210-D011.txt 
             0              0              0              0              0 
S6210-D043.txt S9983-D013.txt 
             0              0

In [38]:
# Check it worked correctly by getting count.
length(list.files())

[1] 915

In [39]:
letters <- lettersFinal
glimpse(letters)

Rows: 915
Columns: 38
$ X.2                     <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…
$ X.1                     <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…
$ X                       <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…
$ docsequence             <int> 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3…
$ docid                   <chr> "S10003-D023", "S10003-D024", "S10003-D025", …
$ docyear                 <int> 1836, 1836, 1837, 1837, 1838, 1838, 1838, 183…
$ doctype                 <chr> "Letter", "Letter", "Letter", "Letter", "Lett…
$ allsubject              <chr> "Childbirth; Church attendance; Cities; Farms…
$ broadsubj               <chr> "Health; Religion; Communities; Relationships…
$ personalevent           <chr> NA, NA, NA, NA, NA, NA, NA, NA, "Physical ill…
$ wwritten                <chr> "Baltimore, MD; Maryland; United States; Mid-…
$ docauthorid             <chr> "per0022938", "per0022938", "per0022938", "pe…
$ docauthorname           <chr

## Writer Location

In [40]:
# New variable for author location
letters$authorLocation <- letters$wwritten

#Recode Canada
rows = which(grepl('Canada', letters$authorLocation)) # Get rows that meet condition
letters$authorLocation[rows] <- "Canada" # Recode data

#Recode United States
rows = which(grepl('United States', letters$authorLocation)) # Get rows that meet condition
letters$authorLocation[rows] <- "United States" # Recode data

#Recode Mexico, Jamaica, Panama, etc.
rows = which(!(grepl('Canada',letters$authorLocation)) &
       !(grepl('United States',letters$authorLocation))) # Get rows that meet condition
letters$authorLocation[rows] <- "Other" # Recode data

# Convert character vars to factor
letters$authorLocation <- as.factor(letters$authorLocation)

#Check counts
summary(letters$authorLocation)

Canada         Other United States 
          427             7           481

## Writer Occupation

In [41]:
# Get a list of unique job titles (note: some cells contain multiple titles)
Job  <- letters$northamericanoccupation %>%
str_split("; ") %>% 
unlist() %>% 
unique() 

Job

[1] "Homemaker"            "Physician"            "Nun"                 
 [4] "Social worker"        "Teacher"              NA                    
 [7] "Jeweler"              "Retail worker"        "Businessman"         
[10] "Factory worker"       "Student"              "Farmer"              
[13] "Servant"              "Tailor"               "Artist"              
[16] "Clergy"               "Missionary"           "Military"            
[19] "Writer"               "Architect"            "Educator"            
[22] "Merchant"             "Banker"               "Engineer"            
[25] "Miner"                "Politician"           "Government appointee"
[28] "Religious leader"     "Tradesman"            "Rancher"             
[31] "Laborer"              "Surveyor"             "Manufacturer"        
[34] "Royal governor"       "Secretary"            "Accountant"          
[37] "Editor"               "Cook"                 "Housekeeper"         
[40] "Nurse"                "Religious worker"     "Explorer"            
[43] "Diplomat"             "Plantation manager"

In [42]:
# Turn this list into a dataframe.
jobClass <- data.frame(Job)
head(jobClass)

Job
Homemaker
Physician
Nun
Social worker
Teacher
NA


In [43]:
# Add a column for the Step 1 (Erickson) classification.
jobClass['Erickson']  <- NA
glimpse(jobClass)

Rows: 44
Columns: 2
$ Job      <fct> Homemaker, Physician, Nun, Social worker, Teacher, NA, Jewel…
$ Erickson <lgl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …


In [44]:
# Convert factors to character.
jobClass  <- unfactorize(jobClass)
glimpse(jobClass)

Rows: 44
Columns: 2
$ Job      <chr> "Homemaker", "Physician", "Nun", "Social worker", "Teacher",…
$ Erickson <lgl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, …


In [45]:
# Make Erickson categories
# Agricultural (A), Industrial (I), Commercial-Clerical-Professional (CCP)

A  <- c("Farmer",
        "Rancher", 
        "Plantation manager",
        "Laborer"# Five out of six migrants categorized only as laborers were in Hawaii, where the main 19th century industries were whaling, sandlewood and sugar. For this reason, I am classifying laborer as agricultural.
       )

I  <-  c("Miner",
         "Manufacturer",
         "Factory worker",
         "Explorer" # Note: This is a prospector so closest to mining in the Erickson classification system.
               ) 

CCP <- c("Jeweler",
                "Tradesman", 
                "Retail worker", 
                "Businessman", 
                "Tailor", 
                "Merchant", 
                "Banker",
                "Physician", 
                   "Architect", 
                   "Engineer", 
                   "Artist", 
                   "Writer", 
                   "Surveyor",
                   "Secretary",
                   "Accountant",
                   "Editor", 
                   "Nurse",
         "Clergy", 
            "Missionary",
            "Educator", 
            "Religious leader",
            "Social worker", 
            "Religious worker",
            "Military", 
            "Politician",
            "Government appointee",
            "Royal governor", 
            "Diplomat",
         "Nun", 
            "Teacher",
         "Servant", 
              "Cook", 
             "Housekeeper")

Other  <- c("Homemaker",
             "Student"
              )

In [46]:
# Map jobs to occupation categories.
# Agricultural
rows = which(grepl(paste(A,collapse="|"), jobClass$Job)) # Get rows that meet condition
jobClass$Erickson[rows] <- "A" # Recode data

# Get Rows where job is in Industry
rows = which(grepl(paste(I,collapse="|"), jobClass$Job))
jobClass$Erickson[rows] <- "I"

# Commercial
rows = which(grepl(paste(CCP,collapse="|"), jobClass$Job))
jobClass$Erickson[rows] <- "CCP"

# Professional
rows = which(grepl(paste(Other,collapse="|"), jobClass$Job))
jobClass$Erickson[rows] <- "Other"

# View
jobClass

Job,Erickson
Homemaker,Other
Physician,CCP
Nun,CCP
Social worker,CCP
Teacher,CCP
NA,NA
Jeweler,CCP
Retail worker,CCP
Businessman,CCP
Factory worker,I


In [47]:
# Take a closer look at certain jobs to figure out how to classify.
unique(subset(letters, grepl('Explorer',northamericanoccupation) | 
              grepl('Artist',northamericanoccupation) | 
              grepl('Laborer',northamericanoccupation) | 
              grepl('Student',northamericanoccupation) | 
              grepl('Homemaker',northamericanoccupation), 
              select=c("docauthorname", "northamericanoccupation")))

,docauthorname,northamericanoccupation
1,"Bruns, Jette, 1813-1899",Homemaker; Physician
209,"Anonymous Russian Jewish Young Male, fl. 1908",Student
216,"Anonymous Russian Jewish Mother, fl. 1910",Homemaker
220,"Anonymous Russian Jewish Male, V. A., fl. 1911",Artist
221,"Anonymous Jewish Male Newsboy, 1897-",Student
339,"Harris, Sarah Stretch, 1818-1897",Architect; Artist; Businessman; Clergy
341,"Harris, Critchlow, 1813-1899",Architect; Artist; Businessman; Clergy; Educator
374,"Harris, Tom, 1847-",Architect; Artist; Businessman; Clergy; Educator; Farmer
384,"Harris, Robert, 1849-1919",Architect; Artist; Businessman; Clergy; Educator; Farmer
414,"Harris, Margaret Ellin, 1851-1944",Architect; Artist; Businessman; Educator; Farmer


In [48]:
# Make categories to break down CCP and other class a bit better.
# Erickson enriched with Miller et al.

Agricultural  <- c("Farmer", # Farmers and planters
                   "Rancher", 
                   "Plantation manager",
                   "Laborer" # Five out of six migrants categorized only as laborers were in Hawaii, where the main 19th century industries were whaling, sandlewood and sugar. For this reason, I am classifying laborer as agricultural.
                  )

Industrial  <-  c("Miner", 
                  "Manufacturer", 
                  "Factory worker") 

Commercial <- c("Jeweler", # Merchants, shopkeepers and peddlers
                "Tradesman", 
                "Retail worker", 
                "Businessman", 
                "Tailor", 
                "Merchant", 
                "Banker",
               "Explorer", # Note: This is a prospector so more like a peddler
               "Artist") # Six out of the seven artists are also architects and businesspeople. The 7th has not other occupation so assuming that he tried to sell his art to make ends meet, thus classifying him as a peddler.

Professional  <- c("Architect", 
                   "Engineer", 
                   "Writer", 
                   "Surveyor",
                   "Secretary",
                   "Accountant",
                   "Editor",
                   "Banker")

Domestic  <- c("Homemaker", # Craftsmen, laborers and servants 
              "Servant", 
              "Cook", 
              "Housekeeper") 

Social  <- c("Nun", #Clergymen and schoolmasters
                "Teacher", 
                "Student", 
                "Clergy", 
                "Missionary", 
                "Educator", 
                "Religious leader", 
                "Social worker", 
                "Religious worker", 
                "Nurse",
                "Physician")

Government  <- c("Military", # Politics, military and Government
                 "Politician", 
                 "Government appointee", 
                 "Royal governor", 
                 "Diplomat")



In [49]:
# Map jobs to categories.
# Agricultural
rows = which(grepl(paste(Agricultural,collapse="|"), jobClass$Job)) # Get rows that meet condition
jobClass$Category[rows] <- "Agricultural" # Recode data

# Industry
rows = which(grepl(paste(Industrial,collapse="|"), jobClass$Job))
jobClass$Category[rows] <- "Industrial"

# Commercial
rows = which(grepl(paste(Commercial,collapse="|"), jobClass$Job))
jobClass$Category[rows] <- "Commercial"

# Professional
rows = which(grepl(paste(Professional,collapse="|"), jobClass$Job))
jobClass$Category[rows] <- "Professional"

# Service
rows = which(grepl(paste(Domestic,collapse="|"), jobClass$Job))
jobClass$Category[rows] <- "Domestic"

# Social
rows = which(grepl(paste(Social,collapse="|"), jobClass$Job))
jobClass$Category[rows] <- "Social"

# Government
rows = which(grepl(paste(Government,collapse="|"), jobClass$Job))
jobClass$Category[rows] <- "Government"

# View
jobClass

Job,Erickson,Category
Homemaker,Other,Domestic
Physician,CCP,Social
Nun,CCP,Social
Social worker,CCP,Social
Teacher,CCP,Social
NA,NA,NA
Jeweler,CCP,Commercial
Retail worker,CCP,Commercial
Businessman,CCP,Commercial
Factory worker,I,Industrial


In [68]:
letters <- unfactorize(letters)

In [66]:
# Step 1
# Agricultural Class ("A")

rows = which(grepl(paste(A,collapse="|"), letters$northamericanoccupation)) # Get rows that meet condition
letters['A'] <- NA # Make binary variable an fill with NAs
letters$A[!is.na(letters$northamericanoccupation)]  <- FALSE # Set non-NA rows to False
letters$A[rows] <- TRUE # Set rows meeting condition to True
summary(letters$A) #Get summary

# Industrial Class ("I")

rows = which(grepl(paste(I,collapse="|"), letters$northamericanoccupation)) # Get rows that meet condition
letters['I'] <- NA # Make binary variable an fill with NAs
letters$I[!is.na(letters$northamericanoccupation)]  <- FALSE # Set non-NA rows to False
letters$I[rows] <- TRUE # Set rows meeting condition to True
summary(letters$I) #Get summary

# Industrial Class ("CCP")

rows = which(grepl(paste(CCP,collapse="|"), letters$northamericanoccupation)) # Get rows that meet condition
letters['CCP'] <- NA # Make binary variable an fill with NAs
letters$CCP[!is.na(letters$northamericanoccupation)]  <- FALSE # Set non-NA rows to False
letters$CCP[rows] <- TRUE # Set rows meeting condition to True
summary(letters$CCP) #Get summary

# Other Class ("Other")

rows = which(grepl(paste(Other,collapse="|"), letters$northamericanoccupation)) # Get rows that meet condition
letters['Other'] <- NA # Make binary variable an fill with NAs
letters$Other[!is.na(letters$northamericanoccupation)]  <- FALSE # Set non-NA rows to False
letters$Other[rows] <- TRUE # Set rows meeting condition to True
summary(letters$Other) #Get summary

   Mode   FALSE    TRUE    NA's 
logical     534     230     151 

   Mode   FALSE    TRUE    NA's 
logical     678      86     151 

   Mode   FALSE    TRUE    NA's 
logical      95     669     151 

   Mode   FALSE    TRUE    NA's 
logical     623     141     151 

In [69]:
# Step 2
# Agricultural Class ("Agricultural")

rows = which(grepl(paste(Agricultural,collapse="|"), letters$northamericanoccupation)) # Get rows that meet condition
letters['Agricultural'] <- NA # Make binary variable an fill with NAs
letters$Agricultural[!is.na(letters$northamericanoccupation)]  <- FALSE # Set non-NA rows to False
letters$Agricultural[rows] <- TRUE # Set rows meeting condition to True
summary(letters$Agricultural) #Get summary

# Industrial Class ("Industrial")

rows = which(grepl(paste(Industrial,collapse="|"), letters$northamericanoccupation)) # Get rows that meet condition
letters['Industrial'] <- NA # Make binary variable an fill with NAs
letters$Industrial[!is.na(letters$northamericanoccupation)]  <- FALSE # Set non-NA rows to False
letters$Industrial[rows] <- TRUE # Set rows meeting condition to True
summary(letters$Industrial) #Get summary

# Commercial Class ("Commercial")

rows = which(grepl(paste(Commercial,collapse="|"), letters$northamericanoccupation)) # Get rows that meet condition
letters['Commercial'] <- NA # Make binary variable an fill with NAs
letters$Commercial[!is.na(letters$northamericanoccupation)]  <- FALSE # Set non-NA rows to False
letters$Commercial[rows] <- TRUE # Set rows meeting condition to True
summary(letters$Commercial) #Get summary

# Professional Class ("Professional")

rows = which(grepl(paste(Professional,collapse="|"), letters$northamericanoccupation)) # Get rows that meet condition
letters['Professional'] <- NA # Make binary variable an fill with NAs
letters$Professional[!is.na(letters$northamericanoccupation)]  <- FALSE # Set non-NA rows to False
letters$Professional[rows] <- TRUE # Set rows meeting condition to True
summary(letters$Professional) #Get summary

# Domestic Class ("Domestic")

rows = which(grepl(paste(Domestic,collapse="|"), letters$northamericanoccupation)) # Get rows that meet condition
letters['Domestic'] <- NA # Make binary variable an fill with NAs
letters$Domestic[!is.na(letters$northamericanoccupation)]  <- FALSE # Set non-NA rows to False
letters$Domestic[rows] <- TRUE # Set rows meeting condition to True
summary(letters$Domestic) #Get summary

# Social Class ("Social")

rows = which(grepl(paste(Social,collapse="|"), letters$northamericanoccupation)) # Get rows that meet condition
letters['Social'] <- NA # Make binary variable an fill with NAs
letters$Social[!is.na(letters$northamericanoccupation)]  <- FALSE # Set non-NA rows to False
letters$Social[rows] <- TRUE # Set rows meeting condition to True
summary(letters$Social) #Get summary

# Government Class ("Government")

rows = which(grepl(paste(Government,collapse="|"), letters$northamericanoccupation)) # Get rows that meet condition
letters['Government'] <- NA # Make binary variable an fill with NAs
letters$Government[!is.na(letters$northamericanoccupation)]  <- FALSE # Set non-NA rows to False
letters$Government[rows] <- TRUE # Set rows meeting condition to True
summary(letters$Government) #Get summary

   Mode   FALSE    TRUE    NA's 
logical     534     230     151 

   Mode   FALSE    TRUE    NA's 
logical     678      86     151 

   Mode   FALSE    TRUE    NA's 
logical     491     273     151 

   Mode   FALSE    TRUE    NA's 
logical     423     341     151 

   Mode   FALSE    TRUE    NA's 
logical     622     142     151 

   Mode   FALSE    TRUE    NA's 
logical     293     471     151 

   Mode   FALSE    TRUE    NA's 
logical     618     146     151 

In [70]:
letters  <- factorize(letters)
glimpse(letters)

Rows: 915
Columns: 50
$ X.2                     <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…
$ X.1                     <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…
$ X                       <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…
$ docsequence             <int> 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3…
$ docid                   <fct> S10003-D023, S10003-D024, S10003-D025, S10003…
$ docyear                 <int> 1836, 1836, 1837, 1837, 1838, 1838, 1838, 183…
$ doctype                 <fct> Letter, Letter, Letter, Letter, Letter, Lette…
$ allsubject              <fct> "Childbirth; Church attendance; Cities; Farms…
$ broadsubj               <fct> Health; Religion; Communities; Relationships;…
$ personalevent           <fct> NA, NA, NA, NA, NA, NA, NA, NA, Physical illn…
$ wwritten                <fct> "Baltimore, MD; Maryland; United States; Mid-…
$ docauthorid             <fct> per0022938, per0022938, per0022938, per002293…
$ docauthorname           <fct

In [71]:
# Get rid of first three columns (remnant index columns)
letters <- letters[c(-1,-2, -3)]
names(letters)

[1] "docsequence"             "docid"                  
 [3] "docyear"                 "doctype"                
 [5] "allsubject"              "broadsubj"              
 [7] "personalevent"           "wwritten"               
 [9] "docauthorid"             "docauthorname"          
[11] "language"                "editor"                 
[13] "briefname"               "nationality"            
[15] "birthyear"               "deathyear"              
[17] "birthplace"              "deathplace"             
[19] "ageatdeath"              "nativeoccupation"       
[21] "yearimmigration"         "stayednorthamerica.x"   
[23] "stayednorthamerica.y"    "agewriting"             
[25] "marriagestatus"          "maternalstatus"         
[27] "authorgeneration"        "authorgender"           
[29] "culturalheritage"        "religion"               
[31] "northamericanoccupation" "religionNew"            
[33] "nationalOrigin"          "culture"                
[35] "sinceMigration"          "authorLocation"         
[37] "A"                       "Agricultural"           
[39] "I"                       "Industrial"             
[41] "CCP"                     "Commercial"             
[43] "Professional"            "Domestic"               
[45] "Social"                  "Government"             
[47] "Other"

In [74]:
# Write a new .csv
write.csv(letters, "../20201219_AM_Meta2Merge.csv")